# Implementing Neural Networks
The World Health Organization (WHO)’s Global Health Observatory (GHO) data repository tracks life expectancy for countries worldwide by following health status and many other related factors.

In this project, we will design, train, and evaluate a neural network model performing the task of regression to predict the life expectancy of countries using this dataset. 

In [79]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from tensorflow.keras import Sequential
from tensorflow.keras.layers import InputLayer,Dense,Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [45]:
df = pd.read_csv('life_expectency.csv')
df.head()

,Country,Year,Status,Adult Mortality,infant deaths,Alcohol,percentage expenditure,Hepatitis B,Measles,BMI,under-five deaths,Polio,Total expenditure,Diphtheria,HIV/AIDS,GDP,Population,thinness 1-19 years,thinness 5-9 years,Income composition of resources,Schooling,Life expectancy
0,Afghanistan,2015,Developing,263.0,62,0.01,71.279624,65.0,1154,19.1,83,6.0,8.16,65.0,0.1,584.259210,33736494.0,17.2,17.3,0.479,10.1,65.0
1,Afghanistan,2014,Developing,271.0,64,0.01,73.523582,62.0,492,18.6,86,58.0,8.18,62.0,0.1,612.696514,327582.0,17.5,17.5,0.476,10.0,59.9
2,Afghanistan,2013,Developing,268.0,66,0.01,73.219243,64.0,430,18.1,89,62.0,8.13,64.0,0.1,631.744976,31731688.0,17.7,17.7,0.470,9.9,59.9
3,Afghanistan,2012,Developing,272.0,69,0.01,78.184215,67.0,2787,17.6,93,67.0,8.52,67.0,0.1,669.959000,3696958.0,17.9,18.0,0.463,9.8,59.5
4,Afghanistan,2011,Developing,275.0,71,0.01,7.097109,68.0,3013,17.2,97,68.0,7.87,68.0,0.1,63.537231,2978599.0,18.2,18.2,0.454,9.5,59.2


In [46]:
print(df.info())
print(df.Status.unique())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2938 entries, 0 to 2937
Data columns (total 22 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Country                          2938 non-null   object 
 1   Year                             2938 non-null   int64  
 2   Status                           2938 non-null   object 
 3   Adult Mortality                  2938 non-null   float64
 4   infant deaths                    2938 non-null   int64  
 5   Alcohol                          2938 non-null   float64
 6   percentage expenditure           2938 non-null   float64
 7   Hepatitis B                      2938 non-null   float64
 8   Measles                          2938 non-null   int64  
 9    BMI                             2938 non-null   float64
 10  under-five deaths                2938 non-null   int64  
 11  Polio                            2938 non-null   float64
 12  Total expenditure   

# Data Preprocessing

In [50]:
features = df.iloc[:,0:-1]
label = df.iloc[:,-1]
features = pd.get_dummies(features)
x_train,x_test,y_train,y_test = train_test_split(features,label,test_size = 0.2,random_state=42)

In [76]:
numerical_features = x_train.select_dtypes(include=['float64', 'int64'])
ct = ColumnTransformer([('scale',StandardScaler(),numerical_features.columns)])
ct.fit_transform(x_train)
ct.transform(x_test)

array([[-0.3334299 , -0.34188905, -0.18596531, ...,  0.33177057,
         0.01166804, -0.1976551 ],
       [-0.3334299 , -0.42186349, -0.25751384, ..., -0.67739751,
         0.73963   ,  0.57075657],
       [-0.11655742, -0.32589416, -0.25751384, ..., -0.58769368,
         0.63563544,  0.47854717],
       ...,
       [ 1.6184224 ,  0.76975572, -0.23068314, ...,  0.53360418,
        -0.90942668, -0.96606677],
       [-0.3334299 ,  0.17794484, -0.25751384, ...,  0.39904844,
         0.58116209,  0.0789731 ],
       [ 0.10031506, -0.26991205, -0.24857028, ..., -0.54284177,
         0.57125785,  0.4478107 ]])

# Building the model

In [83]:
def design_model(features):
  model = Sequential()
  model.add(InputLayer(input_shape = (features.shape[1],)))
  model.add(Dense(50,activation = 'relu'))
  model.add(Dropout(0.2))
  model.add(Dense(10,activation = 'relu'))
  model.add(Dropout(0.1))
  model.add(Dense(10,activation = 'relu'))
  model.add(Dropout(0.1))
  model.add(Dense(1))
  model.compile(loss='mse' ,metrics=['mae'],optimizer =Adam(learning_rate=0.01))
  model.summary()
  return model


In [86]:
es = EarlyStopping(monitor='val_mse',mode='min',patience=5)
model = design_model(x_train)
model.fit(x_train,y_train,batch_size = 2,epochs=100,verbose=1,callbacks=es,validation_split = 0.2)

Model: "sequential_19"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_69 (Dense)            (None, 50)                10750     
                                                                 
 dropout_49 (Dropout)        (None, 50)                0         
                                                                 
 dense_70 (Dense)            (None, 10)                510       
                                                                 
 dropout_50 (Dropout)        (None, 10)                0         
                                                                 
 dense_71 (Dense)            (None, 10)                110       
                                                                 
 dropout_51 (Dropout)        (None, 10)                0         
                                                                 
 dense_72 (Dense)            (None, 1)               

In [87]:
final_mse, final_mae = model.evaluate(x_test, y_test, verbose = 0)
print(final_mse, final_mae)

86.68817901611328 7.712214469909668
